In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../')
from src.EspnFantasyLeague import EspnFantasyLeague
from src.utils.io_json import read_json
from src.utils.utils import simple_matchup_projection

# Input variables

In [2]:
week = 7  # Upcoming round number of the fantasy league to project: CHANGE THIS ACCORDINGLY
start_date = '2021-02-08'  # start date of the week to project: CHANGE THIS ACCORDINGLY
team1 = 'DGGG'  # Abbreviation of fantasy team 1: CHANGE THIS ACCORDINGLY
team2 = 'LAM'  # Abbreviation of fantasy team 2: CHANGE THIS ACCORDINGLY

start_date = pd.to_datetime(start_date)
end_date = start_date + pd.Timedelta(6, 'D')  # Usually a round lasts for a week, if this is not the case, change end data manually.
print(end_date)
week_of_year = start_date.week

2021-02-14 00:00:00


# Fetch Data

In [3]:
settings = read_json('../config/config.json')

In [6]:
cookies = settings['cookies']
league_settings = settings['league']
url_settings = {
    'url_fantasy': settings['url']['fantasy_league'],
    'url_nba': settings['url']['nba']
}

In [7]:
espn_api = EspnFantasyLeague(cookies=cookies,
                             **url_settings,
                             **league_settings)

In [8]:
teams_data = espn_api.get_teams_data()

18 teams participating


In [9]:
schedule_df = espn_api.get_schedule_data(since=start_date, to=end_date)

# Project Upcoming Match-up Stats

In [10]:
fantasy_teams_current_stats_df = espn_api.make_stat_table(teams_data)

In [11]:
fantasy_teams_stats_df = espn_api.get_total_team_stats_upto_round(teams_data, week=week)

## Naive projection
Based on teams' total stats

In [12]:
simple_matchup_projection(fantasy_teams_stats_df, team1, team2)

projected score DGGG-LAM: 3-6-0


,FT%,FG%,PTS,3PM,BLK,STL,AST,REB,TO
teamAbbr,,,,,,,,,
DGGG,0.743,0.4782,3458,402,136,218,846,1224,421
LAM,0.7819,0.4747,3613,437,152,215,698,1268,411
Projected Winner,LAM,DGGG,LAM,LAM,LAM,DGGG,DGGG,LAM,LAM


## Simulation Projection
Based on teams' schedule, players and their mean stats.

In [13]:
espn_api.simulation(teams_data, schedule_df, team1, team2)

,WinProb
FG%,98.804
FT%,8.643
3PM,47.674
REB,59.261
AST,82.853
STL,35.006
BLK,24.276
PTS,43.786
TO,48.636


# Get Weekly Schedule of Match-up
Number of games played by the two fantasy teams in the upcoming round, Injured players are excluded, DTD are included.

In [14]:
schedule_data = espn_api.get_espn_data(espn_api.url_nba, endpoints=['proTeamSchedules'])
teams_schedule_df = espn_api.make_team_scehdule_table(schedule_data)
table, total = espn_api.compare_schedules(teams_data, teams_schedule_df, week_of_year, team1, team2)

In [15]:
table

,Count-DGGG,Count_valid-DGGG,Unused_subs-DGGG,Count-LAM,Count_valid-LAM,Unused_subs-LAM
Date,,,,,,
2021-02-08,7,7,0,6,6,0
2021-02-09,5,5,0,5,5,0
2021-02-10,7,7,0,8,8,0
2021-02-11,3,3,0,2,2,0
2021-02-12,8,8,0,9,8,1
2021-02-13,3,3,0,3,3,0
2021-02-14,8,8,0,8,8,0


In [16]:
total

Count_valid-DGGG    41
Count_valid-LAM     40
dtype: int64

## V2

In [17]:
table2, total2 = espn_api.compare_schedules_2(teams_data, schedule_df, team1, team2)

In [18]:
table2

,Count-DGGG,Count_valid-DGGG,Unused_subs-DGGG,Count-LAM,Count_valid-LAM,Unused_subs-LAM
date,,,,,,
2021-02-08,7,7,0,6,6,0
2021-02-09,5,5,0,5,5,0
2021-02-10,7,7,0,8,8,0
2021-02-11,3,3,0,2,2,0
2021-02-12,8,8,0,9,8,1
2021-02-13,3,3,0,3,3,0
2021-02-14,8,8,0,8,8,0


In [19]:
total2

Count_valid-DGGG    41
Count_valid-LAM     40
dtype: int64

In [20]:
table.equals(table2)

True

In [21]:
total.equals(total2)

True